In [4]:
!sudo -H pip install -q transformers --upgrade

In [7]:
!sudo -H pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.7/autoawq-0.1.7+cu118-cp38-cp38-linux_x86_64.whl

     |████████████████████████████████| 20.0MB 6.6MB/s eta 0:00:01
     |████████████████████████████████| 184kB 13.6MB/s eta 0:00:01
     |████████████████████████████████| 81kB 14.1MB/s eta 0:00:01
     |████████████████████████████████| 163kB 17.3MB/s eta 0:00:01
     |████████████████████████████████| 235kB 21.7MB/s eta 0:00:01


     |████████████████████████████████| 225kB 19.8MB/s eta 0:00:01
     |████████████████████████████████| 235kB 19.7MB/s eta 0:00:01
     |████████████████████████████████| 10.1MB 38.7MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 71kB 40.3MB/s  eta 0:00:01
     |████████████████████████████████| 112kB 110.8MB/s eta 0:00:01
     |████████████████████████████████| 389kB 112.8MB/s eta 0:00:01
     |████████████████████████████████| 204kB 99.5MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 94.1MB/s eta 0:00:01
     |████████████████████████████████| 51kB 32.9MB/s eta 0:00:01


     |████████████████████████████████| 81kB 46.7MB/s  eta 0:00:01
     |████████████████████████████████| 389kB 89.5MB/s eta 0:00:01
     |████████████████████████████████| 1.5MB 83.0MB/s eta 0:00:01
     |████████████████████████████████| 471kB 86.3MB/s eta 0:00:01
     |████████████████████████████████| 92kB 47.9MB/s  eta 0:00:01
     |████████████████████████████████| 81kB 43.2MB/s  eta 0:00:01
     |████████████████████████████████| 2.1MB 84.0MB/s eta 0:00:01
     |████████████████████████████████| 61kB 39.2MB/s eta 0:00:01
  Created wheel for pycountry: filename=pycountry-22.3.5-cp38-none-any.whl size=10681832 sha256=aa5ee3c949c18fb456164988f4b477b82194ab3a9a5bbdacdea4ec471e49d2df
  Stored in directory: /root/.cache/pip/wheels/7a/c3/f8/3e164430062ee08d5b8bc8fcd1b891ce54c7ad99871aa3adc1
Successfully built pycountry
  Created wheel for rouge-score: filename=rouge_score-0.1.2-cp38-none-any.whl size=24937 sha256=428f217f69b7db0f7583b58b4f24a694b24e689695ad2c23bd805ad8a40039e6
  Store

  Found existing installation: platformdirs 3.6.0
    Uninstalling platformdirs-3.6.0:
      Successfully uninstalled platformdirs-3.6.0
  Found existing installation: pluggy 1.0.0
    Uninstalling pluggy-1.0.0:
      Successfully uninstalled pluggy-1.0.0


In [1]:
import pandas

In [2]:
import numpy

In [1]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

In [1]:
!pip list | grep cuda

nvidia-cuda-cupti-cu11   11.7.101  
nvidia-cuda-nvrtc-cu11   11.7.99   
nvidia-cuda-runtime-cu11 11.7.99   


In [2]:
import pandas

In [4]:
import torch
torch.__version__

'2.0.1+cu117'

In [ ]:
model_path = '/data/quantization-trials/merged-model'

In [ ]:
quant_path = '/data/quantization-trials/quant-model'

In [7]:
quant_config = {"zero_point":True,
               "q_group_size": 512,
               "w_bit": 4,
               "version": "GEMM"
               }

In [28]:
#load model
# model = AutoAWQForCausalLM.from_pretrained(model_path, **{"low_cpu_mem_usage":True})
model = AutoAWQForCausalLM.from_pretrained(model_path, safetensors=True, device_map='auto')

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [10]:
#Quantize
# dont run it again
model.quantize(tokenizer, quant_config=quant_config)

Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

AWQ: 100%|██████████| 24/24 [06:42<00:00, 16.76s/it]


In [13]:
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

('/data/quantization-trials/quant-model/tokenizer_config.json',
 '/data/quantization-trials/quant-model/special_tokens_map.json',
 '/data/quantization-trials/quant-model/vocab.json',
 '/data/quantization-trials/quant-model/merges.txt',
 '/data/quantization-trials/quant-model/added_tokens.json',
 '/data/quantization-trials/quant-model/tokenizer.json')

### Inference on quantized model

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
quant_path = '/data/quantization-trials/quant-model'
model_path = '/data/quantization-trials/merged-model'

In [7]:
model_id = quant_path
quant_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

def predict_from_quant(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = quant_model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

In [8]:
import time

In [9]:
# Using quant model
start = time.time()
output1 = predict_from_quant("what is science")
print("time taken is :", time.time()-start)

time taken is : 22.486776113510132


In [13]:
print(output1)

</s>what is science?

Science is a method of understanding the world and the world around us. It is a way of thinking about the world and the world around us. It is a way of understanding the world and the world around us.

What is the world?

The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in the world. The world is a collection of things that are in 

### Inference on un-quantized model

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
model_path = '/data/quantization-trials/merged-model'

In [3]:
model_id = model_path
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

def predict_from_normal(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

In [4]:
# Using unquant model
import time
start = time.time()
output2 = predict_from_normal("what is science")
print("time taken is :", time.time()-start)

time taken is : 24.060552835464478


In [48]:
print(output2)

</s>what is science?

The word "science" is used to describe a variety of fields of study, including:

biology

chemistry

biology of the brain

biology of the immune system

biology of the nervous system

biology of the reproductive system

biology of the immune system

biology of the nervous system

biology of the reproductive system

biology of the immune system

biology of the reproductive system

biology of the nervous system

biology of the reproductive system

biology of the immune system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive sy

In [36]:
model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
          (final_layer_norm): LayerN

In [33]:
quant_model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (v_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (q_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (out_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): WQLinear_GEMM(in_features=2048, out